In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense

from connections import password
from Playlist_Track_Features import playlist_track_features

C:\Users\Will\Anaconda3\envs\PythonData\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
playlist_features = playlist_track_features('Everything Good', 'Inger Laisy Niemi')

 playlist_id: 07eLdS5uGbyK6Gb7bF8VaF | user_id: laisy


In [3]:
# Import features for scaling 
scale_features = pd.read_csv('Database_Setup/features.csv')
scale_features.drop(['Unnamed: 0', 'ID'], axis = 1, inplace = True)
playlist_features.rename(columns = {'acousticness':'Acousticness', 'danceability':'Danceability', 
                    'duration_ms':'Duration_ms', 'energy':'Energy', 
                    'id':'ID', 'instrumentalness':'Instrumentalness', 
                    'key':'Key', 'liveness':'Liveness', 
                    'loudness':'Loudness', 'mode':'Mode', 
                    'speechiness':'Speechiness', 'tempo':'Tempo', 
                    'time_signature':'Time_Signature', 'valence':'Valence'}, inplace = True)


In [4]:
playlist_features['label'] = True
scale_features['label'] = False

In [5]:
all_features = playlist_features.append(scale_features)

In [6]:
X = all_features.drop(['label'], axis = 1)
y = all_features['label']

In [7]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state = 1, stratify = y)
X_scaler = StandardScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Label-encode dataset
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

# Convert encoded labels to one-hot-encodeing
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

In [8]:
def build_model():
    model = Sequential()
    model.add(Dense(units=5, activation='relu', input_dim=13))
    model.add(Dense(units=3, activation='relu'))
    model.add(Dense(units=2, activation='softmax'))

    # Compile and fit the model
    model.compile(optimizer = 'adam',
                 loss = 'categorical_crossentropy',
                 metrics = ['accuracy'])
    model.fit(X_train, 
             y_train_categorical,
             epochs = 5,
             shuffle = True,
             verbose = 2)
    
    model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=0)
    
    return model, model_accuracy

In [18]:
runs = 0
model, model_accuracy = build_model()
while model_accuracy < 0.9:
    print(model_accuracy)
    print(f"Number of runs: {runs}")
    model, model_accuracy = build_model()
    runs += 1

print(f"Accuracy: {model_accuracy}")
print(f"Number of runs: {runs}")

Epoch 1/5
 - 1s - loss: 0.1364 - acc: 0.9915
Epoch 2/5
 - 0s - loss: 0.1364 - acc: 0.9915
Epoch 3/5
 - 0s - loss: 0.1364 - acc: 0.9915
Epoch 4/5
 - 0s - loss: 0.1364 - acc: 0.9915
Epoch 5/5
 - 0s - loss: 0.1364 - acc: 0.9915
0.7370558376643057
Number of runs: 0
Epoch 1/5
 - 1s - loss: 0.5744 - acc: 0.9915
Epoch 2/5
 - 0s - loss: 0.3955 - acc: 0.9915
Epoch 3/5
 - 0s - loss: 0.2834 - acc: 0.9915
Epoch 4/5
 - 0s - loss: 0.2121 - acc: 0.9915
Epoch 5/5
 - 0s - loss: 0.1655 - acc: 0.9915
Accuracy: 0.9915397631133672
Number of runs: 1


Normal Neural Network - Loss: 0.4732095663317569, Accuracy: 0.9915397631133672


Predicted classes: [False False False False]
Actual Labels:     [False, False, False, False]


C:\Users\Will\Anaconda3\envs\PythonData\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
